In [2]:
import glob, os, re
import pandas as pd
import numpy as np 
from obspy import UTCDateTime
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk

In [3]:
figpath = '/Volumes/home/Research/Detect_fish/Taoyuan_fishcall/fig'
path = '../Results/Detection_fishcall/manual_csv_0.75/202306'

In [4]:
class ImageDisplay:
    def __init__(self, root, image_paths):
        self.root = root
        self.image_paths = image_paths
        self.current_image_index = 0
        self.current_question_index = 0
        self.user_responses = None  
        self.user_peak_responses = None 

        self.load_image(self.current_image_index)
        
        self.root.bind('<Key>', self.on_key_press)

        self.button_frame = tk.Frame(root)
        self.button_frame.pack()

        self.waveform_button = tk.Button(self.button_frame, text="waveform (w)", command=self.on_waveform)
        self.waveform_button.pack(side=tk.LEFT, padx=10)

        self.correlate_button = tk.Button(self.button_frame, text="correlate (c)", command=self.on_correlate)
        self.correlate_button.pack(side=tk.RIGHT, padx=10) 

        self.A_button = tk.Button(self.button_frame, text="A/single frequency (Left)", command=self.on_A)
        self.A_button.pack(side=tk.LEFT, padx=20)

        self.B_button = tk.Button(self.button_frame, text="B/double frequency (Right)", command=self.on_B)
        self.B_button.pack(side=tk.LEFT, padx=10)

        self.C_button = tk.Button(self.button_frame, text="C/interaction (Up)", command=self.on_C)
        self.C_button.pack(side=tk.RIGHT, padx=10)

        self.D_button = tk.Button(self.button_frame, text="D/others (Down)", command=self.on_D)
        self.D_button.pack(side=tk.RIGHT, padx=20)

        self.display_question(self.current_question_index)
        
    def load_image(self, index):
        try:
            image_path = self.image_paths[index]
            image = Image.open(image_path)
            image = image.resize((800, 600), Image.Resampling.LANCZOS) 
            self.photo = ImageTk.PhotoImage(image)
            if hasattr(self, 'label'):
                self.label.config(image=self.photo)
            else:
                self.label = tk.Label(self.root, image=self.photo)
                self.label.pack()
        except:
            print(self.image_paths)
            self.user_responses = 'problem'
            self.user_peak_responses = 'problem'
            self.end_loop()
            pass

    def display_question(self, question_index):
        questions = ["peak waveform (w)/ correlate (c)? ", "Which category ?"]
        if hasattr(self, 'question_label'):
            self.question_label.config(text=questions[self.current_question_index])
        else:
            self.question_label = tk.Label(self.root, text=questions[self.current_question_index])
            self.question_label.pack()
            
        if self.current_question_index == 0:
            self.show_w_c_buttons()
        else:
            self.show_a_b_c_d_buttons()

    def show_w_c_buttons(self):
        self.waveform_button.pack(side=tk.LEFT, padx=10)
        self.correlate_button.pack(side=tk.RIGHT, padx=10)
        self.A_button.pack_forget()
        self.B_button.pack_forget()
        self.C_button.pack_forget()
        self.D_button.pack_forget()

    def show_a_b_c_d_buttons(self):
        self.waveform_button.pack_forget()
        self.correlate_button.pack_forget()
        self.A_button.pack(side=tk.LEFT, padx=20)
        self.B_button.pack(side=tk.LEFT, padx=10)
        self.C_button.pack(side=tk.RIGHT, padx=10)
        self.D_button.pack(side=tk.RIGHT, padx=20)            
            

    def on_A(self):
        self.user_responses = 'A'  # 将 True 写入列表
        self.next_question_or_image()
        # self.current_image_index = (self.current_image_index + 1) % len(self.image_paths)  # 循环更新索引
        # if self.current_image_index == 0:
        #     self.end_loop()  # 如果回到第一张图片，结束循环
        # else:
        #     self.load_image(self.current_image_index)

    def on_B(self):
        self.user_responses = 'B'  # 将 False 写入列表
        self.next_question_or_image()
        # self.current_image_index = (self.current_image_index + 1) % len(self.image_paths)  # 循环更新索引
        # if self.current_image_index == 0:
        #     self.end_loop()  # 如果回到第一张图片，结束循环
        # else:
        #     self.load_image(self.current_image_index)

    def on_C(self):
        self.user_responses = 'C' # 将 False 写入列表
        self.next_question_or_image()
        # self.current_image_index = (self.current_image_index + 1) % len(self.image_paths)  # 循环更新索引
        # if self.current_image_index == 0:
        #     self.end_loop()  # 如果回到第一张图片，结束循环
        # else:
        #     self.load_image(self.current_image_index)
    def on_D(self):
        self.user_responses = 'D' # 将 False 写入列表
        self.next_question_or_image()
        # self.current_image_index = (self.current_image_index + 1) % len(self.image_paths)  # 循环更新索引
        # if self.current_image_index == 0:
        #     self.end_loop()  # 如果回到第一张图片，结束循环
        # else:
        #     self.load_image(self.current_image_index)
    def on_other(self):
        self.user_responses = 'problem' # 将 False 写入列表
        self.next_question_or_image()
#         if self.current_image_index == 0:   
#             self.end_loop()
            
    def on_waveform(self):
        self.user_peak_responses = 'waveform'
        self.next_question_or_image()
    def on_correlate(self):
        self.user_peak_responses = 'corr'
        self.next_question_or_image()

            
    def end_loop(self):
        print(self.user_responses, self.user_peak_responses)
        self.root.quit()  # 退出主循环
        
        return self.user_responses, self.user_peak_responses

    def next_question_or_image(self):
        self.current_question_index += 1
        if self.current_question_index >= 2:  # 如果已经问了两个问题
            self.current_question_index = 0
            self.end_loop()
            self.current_image_index = 0 
            # self.current_image_index = (self.current_image_index + 1) #% len(self.image_paths)  # 循环更新索引
            # if self.current_image_index == 0:
            #     self.end_loop()  # 如果回到第一张图片，结束循环
            # else:
            #     self.load_image(self.current_image_index)
        self.display_question(self.current_question_index)
        
    def on_key_press(self, event):
        if self.current_question_index == 0 and event.keysym.lower() in ['w', 'c']:
            if event.keysym.lower() == 'w':  
                self.on_waveform()
            elif event.keysym.lower() == 'c': 
                self.on_correlate()
        elif self.current_question_index == 1 and event.keysym in ['Left', 'Right', 'Up', 'Down']:
            if event.keysym == 'Left':  
                self.on_A()
            elif event.keysym == 'Right': 
                self.on_B()
            elif event.keysym == 'Up':
                self.on_C()
            elif event.keysym == 'Down': 
                self.on_D()
            else:
                self.on_other()
                print(f"Pressed {event.keysym}")  # 打印按键信息


In [6]:
for csvpath in sorted(glob.glob(f'{path}/*0622*_stats.csv')):
    print(csvpath)
    df = pd.read_csv(csvpath)
    print(len(df[df['keep']==True]))
    responselist = {'peak_method':[] ,'classify':[]} 
    for i in range( len(df)):
        keep = df['keep'].values[i]
        if keep:
            begintmp = re.split('[(|)|,]',df['time_min_date'].values[i])
            tr_begin = UTCDateTime(int(begintmp[1]),int(begintmp[2]),int(begintmp[3]),int(begintmp[4]),int(begintmp[5]),int(begintmp[6]),int(begintmp[7]))
            mm = str(tr_begin.month).zfill(2)
            dd = str(tr_begin.day).zfill(2)
            hh = str(tr_begin.hour).zfill(2)
            minn = str(tr_begin.minute).zfill(2)
            ss = str(tr_begin.second).zfill(2)
            pngfile = glob.glob(f'{figpath}/23{mm}{dd}/??????_{hh}{minn}{ss}.png')
            
            root = tk.Tk()
            root.title("Image Display")
            root.geometry("820x750+100+100")
            image_display = ImageDisplay(root, pngfile)

            # 运行主循环
            root.mainloop()
            response = image_display.user_responses
            peak_method = image_display.user_peak_responses
            responselist['classify'].append(response)
            responselist['peak_method'].append(peak_method)
            
            
            root.destroy()


        else:
            responselist['classify'].append(np.nan)
            responselist['peak_method'].append(np.nan)
        if i%100==0 and i >0:
            tempdf = pd.concat([df, pd.DataFrame(responselist)], axis=1)
            tempdf.to_csv(f"{path}/{re.split('[/|.]',csvpath)[-2]}_classify_temp.csv", index=False)            
    print('finish .........')
    newdf = pd.concat([df, pd.DataFrame(responselist)], axis=1)
    newdf.to_csv(f"{path}/{re.split('[/|.]',csvpath)[-2]}_classify.csv", index=False)
    print('save .........')
   

../Results/Detection_fishcall/manual_csv_0.75/202306/result_230622_stats.csv
349
A corr
B waveform
B waveform
B corr
B waveform
A waveform
A corr
A waveform
B corr
A corr
A corr
B corr
A waveform
A waveform
A waveform
A waveform
C waveform
A waveform
C waveform
A waveform
A waveform
A waveform
A waveform
C waveform
B waveform
A waveform
C waveform
C waveform
B waveform
B waveform
A waveform
A waveform
A waveform
A corr
C waveform
C waveform
C waveform
C waveform
A waveform
C waveform
C corr
A waveform
A waveform
A waveform
A waveform
A waveform
A waveform
A waveform
A waveform
B waveform
C waveform
B waveform
C waveform
B waveform
B waveform
B waveform
B waveform
B waveform
B waveform
B waveform
A waveform
A waveform
A waveform
A corr
C waveform
A waveform
C waveform
A waveform
A waveform
A waveform
A waveform
A waveform
A waveform
A waveform
A waveform
C waveform
C waveform
B waveform
A waveform
C waveform
C waveform
C waveform
C waveform
C waveform
C waveform
C waveform
C waveform
B 